In [1]:
import pandas as pd
import xmlrpc.client
import re


In [2]:
username = "juan.cano@donsson.com"
password = "1000285668"
url = "https://donsson.com"
db = "Donsson_produccion"

common = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/common")
uid = common.authenticate(db, username, password, {})
models = xmlrpc.client.ServerProxy(f"{url}/xmlrpc/2/object")

# 1. Obtener las órdenes de producción ya cerradas (hechas por manufactura)
ordenes_produccion = models.execute_kw(
    db, uid, password,
    'mrp.production', 'search_read',
    [[('state', '=', 'done')]],   # Solo cerradas
    {'fields': ['id','name'#'date_start'
                , 'product_id', 'product_qty','valor_unitario_mrp','date_finished']}
)


ops = pd.DataFrame(ordenes_produccion) # todas las OP cerradas

ops["date_finished"] = pd.to_datetime(ops["date_finished"])

ops = ops[ops["product_qty"]>2] #Esta regla elimina todos los prototipos o op's donde se fabrica una unica unidad

# 1) Quedarnos con el nombre del producto dentro de product_id
#    (Odoo devuelve [id, "nombre"])
ops['product_name_raw'] = ops['product_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v)
)

# 2) Quitar el código entre corchetes "[...]" + espacios iniciales
# 3) Quitar todo lo que esté dentro de paréntesis "(...)" + espacios
ops['producto'] = (
    ops['product_name_raw']
      .astype(str)
      .str.replace(r'^\s*\[.*?\]\s*', '', regex=True)   # elimina "[CODE] "
      .str.replace(r'\(.*?\)', '', regex=True)          # elimina "(...)"
      .str.strip()
)

# (opcional) normalizar espacios internos
ops['producto'] = ops['producto'].str.replace(r'\s+', ' ', regex=True)

# Tu regex original (funciona para códigos alfanuméricos)
ops['product_ref'] = ops["product_name_raw"].str.extract(r"\[([A-Z0-9]+)\]")

# Extraer carcasas (captura lo que tenga letras, números y espacios dentro de corchetes)
ops['carcasa_ref'] = ops["product_name_raw"].str.extract(r"\[(CARCASA.*?)\]")

# Combinar: si no hay product_ref, usar carcasa_ref
ops['product_ref'] = ops['product_ref'].fillna(ops['carcasa_ref'])

# Eliminar la columna auxiliar
ops = ops.drop(columns=['carcasa_ref'])




tiempo_estimado_estandard = models.execute_kw(
    db, uid, password,
    'mrp.routing.workcenter', 'search_read',
    [[]],   # Solo cerradas
    {'fields': ['routing_id','total_nbr_minimo','display_name']}
)

test = pd.DataFrame(tiempo_estimado_estandard)

test['producto'] = test['routing_id'].apply(
    lambda v: v[1] if isinstance(v, (list, tuple)) and len(v) >= 2 else str(v)
)

ruta = "/home/donsson/proyectos/PRORUDCCION/datasets/pareto_produccion.xlsx"
pareto_prod = pd.read_excel(ruta)

pareto_prod["producto"] = (
    pareto_prod["product_name"]
    .str.split("]").str[-1]   # toma lo que viene después de ']'
    .str.strip()              # quita espacios en blanco iniciales
)

pareto_merge = pareto_prod[["producto","Clasificacion"]]

#Demora 1 minuto

KeyboardInterrupt: 

In [ ]:
pareto_prod.head()

In [ ]:
test.head()

In [ ]:
test = test[["producto","display_name","total_nbr_minimo"]]
test_clss = pareto_merge.merge(test , on ="producto" ,how="right")

In [ ]:
test_clss.sample(20)

In [ ]:
print(f"Primera fecha encontrada",ops["date_finished"].min())
print(f"Ultima fecha encontrada",ops["date_finished"].max())

print("_________________________________________________________")
print("")
ops.info()

In [ ]:
ops_def = ops[["name","product_ref","producto","valor_unitario_mrp","product_qty","date_finished"]]
ops_def.head()


In [ ]:
import pandas as pd
import numpy as np


ops_def = ops_def.copy()

# Asegúrate que las fechas estén en datetime
ops_def["date_finished"] = pd.to_datetime(ops_def["date_finished"])

# Ordenar por producto y fecha
ops_sorted = ops_def.sort_values(["product_ref", "date_finished"])


ops_sorted['es_ultimo'] = ops_sorted.groupby('product_ref')['date_finished'].transform('max') == ops_sorted['date_finished']

# 2️⃣ Crear las columnas del 1 anterior
ops_sorted["costo_anterior"] = ops_sorted.groupby("product_ref")["valor_unitario_mrp"].shift(1)
ops_sorted["op_anterior"]    = ops_sorted.groupby("product_ref")["name"].shift(1)
ops_sorted["qty_anterior"]   = ops_sorted.groupby("product_ref")["product_qty"].shift(1)
ops_sorted["fecha_anterior"] = ops_sorted.groupby("product_ref")["date_finished"].shift(1)

# 3️⃣ Crear las columnas del 2 anteriores
ops_sorted["costo_anterior_2"] = ops_sorted.groupby("product_ref")["valor_unitario_mrp"].shift(2)
ops_sorted["op_anterior_2"]    = ops_sorted.groupby("product_ref")["name"].shift(2)
ops_sorted["qty_anterior_2"]   = ops_sorted.groupby("product_ref")["product_qty"].shift(2)
ops_sorted["fecha_anterior_2"] = ops_sorted.groupby("product_ref")["date_finished"].shift(2)

# 4️⃣ Variaciones con el 1 anterior (manteniendo los nombres originales)
ops_sorted["variacion"] = ops_sorted["valor_unitario_mrp"] - ops_sorted["costo_anterior"]
ops_sorted["variacion_abs"] = np.abs(ops_sorted["valor_unitario_mrp"] - ops_sorted["costo_anterior"])
ops_sorted["variacion_pct"] = np.where(
    ops_sorted["costo_anterior"] == 0,
    np.nan,
    (ops_sorted["variacion_abs"] / ops_sorted["costo_anterior"])
)

# 5️⃣ Variaciones con el 2 anterior (con sufijo _2)
ops_sorted["variacion_2"] = ops_sorted["valor_unitario_mrp"] - ops_sorted["costo_anterior_2"]
ops_sorted["variacion_abs_2"] = np.abs(ops_sorted["valor_unitario_mrp"] - ops_sorted["costo_anterior_2"])
ops_sorted["variacion_pct_2"] = np.where(
    ops_sorted["costo_anterior_2"] == 0,
    np.nan,
    (ops_sorted["variacion_abs_2"] / ops_sorted["costo_anterior_2"])
)


# impacto firmado (puede ser positivo o negativo)
ops_sorted["impacto_signed"] = (ops_sorted["valor_unitario_mrp"] - ops_sorted["costo_anterior"]) * ops_sorted["product_qty"]
# impacto firmado (puede ser positivo o negativo)
ops_sorted["impacto_signed_2"] = (ops_sorted["valor_unitario_mrp"] - ops_sorted["costo_anterior_2"]) * ops_sorted["product_qty"]



# impacto absoluto (valor absoluto en pesos, usando qty actual)
ops_sorted["impacto_abs"] = ops_sorted["variacion_abs"] * ops_sorted["product_qty"]
ops_sorted["impacto_abs_2"] = ops_sorted["variacion_abs_2"] * ops_sorted["product_qty"]






def flag_variacion(pct, abs_val):
    if pd.isna(pct):
        return "Sin histórico"
    if pct > 20 or abs_val > 5000000:   # ejemplo: 20% o más de $5MM
        return "Rojo"
    if pct > 5 or abs_val > 1000000:    # ejemplo: entre 5% y 20% o más de $1MM
        return "Amarillo"
    return "Verde"

ops_sorted["flag_var1"] = ops_sorted.apply(lambda r: flag_variacion(r["variacion_pct"], r["impacto_abs"]), axis=1)
ops_sorted["flag_var2"] = ops_sorted.apply(lambda r: flag_variacion(r["variacion_pct_2"], r["impacto_abs_2"]), axis=1)



ops_sorted = ops_sorted.merge(pareto_merge , on ="product_ref" ,how="left")



# Exportar a Excel
ops_sorted.to_excel("/home/donsson/proyectos/PRORUDCCION/datasets/historial_costos_con_variacion.xlsx", index=False)

print("✅ Archivo generado: historial_costos_con_variacion.xlsx")

#Tarda 20 seg


In [ ]:
ops_sorted.sample(20)